In [1]:
import csv
import ipyparams
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import random
import re
import shutil
import string  
import sys
import tensorflow as tf
import tensorflow_hub as hub

from collections import Counter
from google_trans_new import google_translator
from joblib import Parallel, delayed
from nltk.corpus import stopwords
from scipy import stats
from sklearn.feature_extraction import text
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, models, layers, regularizers, preprocessing, datasets, metrics, losses, optimizers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorboard.plugins.hparams import api as hp 

base_path = os.path.abspath('/tf/eipi10/jian-xu3/snps-classification')
sys.path.append(base_path)

import snps 

from snps.core import text_predictor 
from snps.core import util 
from snps.utils import data
from snps.core.text_classification import lr_schedule
from snps.core.text_classification import plot_distribution, plot_length_distribution, plot_frequency_distribution
from snps.core.text_classification import Params, TextClassificationHelper
from snps.core.text_classification import SimpleTextDatasets, SequenceTextDatasets, RawTextDatasets,TransferTextDatasets

<IPython.core.display.Javascript object>

2021-04-22 18:29:58,183: INFO: deployment = dev
2021-04-22 18:29:58,183: INFO: dataset_name = snps_18510_c8
2021-04-22 18:29:58,184: INFO: working_path = /tf/eipi10/jian-xu3/snps-classification


In [2]:
snps.env.params

{'deployment': 'dev',
 'dataset_name': 'snps_18510_c8',
 'document_client_id': 'c9a0e93c-8592-4bfd-b9c3-08cc93480ab2',
 'document_client_secret': 'coL0KKIJUZBCQP9gDrThcr7V3xVl3kA4CZj2Be/cj7g=',
 'list_client_id': 'c03d016f-f430-402a-ad06-c1a021ba79db',
 'list_client_secret': 'ipHd2JgOi0w5W1uckFvGNpIVR86k7kOV4oCEzTOcLaA='}

In [43]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.utils.extmath import stable_cumsum

row_count=5
label_count=2
np.random.seed(2021)

y_true = np.random.randint(0, 2, (row_count, label_count))
y_score = np.random.rand(row_count, label_count)*7/10 + y_true*3/10
y_pred = np.random.randint(0, 2, (row_count, label_count))

print('-'*60)
print(y_true)
# print('-'*60)
# print(y_score.shape, y_score)
print('-'*60)
print( y_pred)

------------------------------------------------------------
[[0 1]
 [1 0]
 [1 0]
 [0 0]
 [1 0]]
------------------------------------------------------------
[[0 1]
 [1 1]
 [0 0]
 [0 1]
 [0 1]]


In [42]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_true, y_pred))

scores = []
for j in range(label_count):
    scores.append(accuracy_score(y_true[:,j], y_pred[:,j]))
print(scores)
np.mean(scores) 

0.2
[0.6, 0.4]


0.5

In [8]:
for average in ['macro', 'micro', 'weighted']:
    print('-'*30, average, '-'*30)    
    print(roc_auc_score(y_true, y_score, average=average))

    scores = []
    weights = []
    
    if average=='macro':
        for j in range(label_count):
            scores.append(roc_auc_score(y_true[:,j], y_score[:,j]))
            weights.append(1)
    elif average=='weighted':
        for j in range(label_count):
            scores.append(roc_auc_score(y_true[:,j], y_score[:,j]))
            weights.append(np.sum(y_true[:,j]))            
    elif average=='micro':        
        scores.append(roc_auc_score(y_true.ravel(), y_score.ravel()))
        weights.append(1)

    scores = np.array(scores)
    weights = np.array(weights)/np.sum(weights)   
    if average=='weighted':
        weighted = weights
    print(np.sum(scores*weights))    

------------------------------ macro ------------------------------
0.8343839848445157
0.8343839848445158
------------------------------ micro ------------------------------
0.8343941642840822
0.8343941642840822
------------------------------ weighted ------------------------------
0.8343397110348449
0.8343397110348447


In [9]:
import tensorflow as tf
num_thresholds = 200 
m = tf.keras.metrics.AUC(num_thresholds=num_thresholds, multi_label=True, label_weights=weighted)
m.update_state(y_true, y_score)
print(m.result().numpy())

m = tf.keras.metrics.AUC(num_thresholds=num_thresholds, multi_label=False)
m.update_state(y_true, y_score)
print(m.result().numpy()) 

0.83571875
0.83560884


In [10]:
def plot_roc_curve(fpr, tpr):
    plt.figure(figsize=(6, 6))    
    plt.title('ROC Curve')
    plt.xlim((-0.01, 1.01))
    plt.ylim((-0.01, 1.01)) 
    plt.ylabel('TPR')
    plt.xlabel('FPR')
    plt.xticks(np.linspace(0, 1, 11))
    plt.yticks(np.linspace(0, 1, 11))
    plt.plot(fpr, tpr)
    
    plt.show()
    
plot_roc_curve(fpr, tpr)

NameError: name 'fpr' is not defined

In [ ]:
print(np.trapz(tpr, fpr))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true[:,0], y_score[:,0], pos_label=1) 
print('-'*30, 'fpr', '-'*30)
print(fpr, len(fpr))
print('-'*30, 'tpr', '-'*30)
print(tpr, len(tpr))
print('-'*30, 'thresholds', '-'*30)
print(thresholds, len(thresholds))  

In [10]:
labels = np.array([0, 1, 3 ,2 ])
new_labels = np.zeros((labels.size, labels.max() + 1))
new_labels[np.arange(labels.size), labels] = 1
new_labels

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [24]:
y_pred = 

(5000, 5)

In [135]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras import metrics
from sklearn.metrics import accuracy_score
import numpy as np

X = [[1,0,0,0,0,0,0],
      [1,2,0,0,0,0,0],
      [3,0,0,0,0,0,0],
      [3,4,0,0,0,0,0],
      [2,0,0,0,0,0,0],
      [3,0,0,0,0,0,0],
      [4,0,0,0,0,0,0],
      [2,3,0,0,0,0,0],
      [1,2,3,0,0,0,0],
      [1,2,3,4,0,0,0],
      [0,0,0,0,0,0,0],
      [1,1,2,3,0,0,0],
      [2,3,3,4,0,0,0],
      [4,4,1,1,2,0,0],
      [1,2,3,3,3,3,3],
      [2,4,2,4,2,0,0],
      [1,3,3,3,0,0,0],
      [4,4,0,0,0,0,0],
      [3,3,0,0,0,0,0],
      [1,1,4,0,0,0,0],
      [3,3,0,1,0,0,0],
      [1,1,4,0,2,0,0]
    ]

Y = [[1,0,0,0],
    [1,1,0,0],
    [0,0,1,0],
    [0,0,1,1],
    [0,1,0,0],
    [0,0,1,0],
    [0,0,0,1],
    [0,1,1,0],
    [1,1,1,0],
    [1,1,1,1],
    [0,0,0,0],
    [1,1,1,0],
    [0,1,1,1],
    [1,1,0,1],
    [1,1,1,0],
    [0,1,0,0],
    [1,0,1,0],
    [0,0,0,1],
    [0,0,1,0],
    [1,0,0,1],
    [0,0,1,0],
    [1,0,0,1]
    ]

X = np.array(X)
Y = np.array(Y)
print(X.shape, Y.shape)

MAXLEN = 7
MAXFEATURES = 4
NUM_CLASSES = 4
model = Sequential()
model.add(Embedding(MAXFEATURES+1,
                    50,
                    input_length=MAXLEN))
model.add(GlobalAveragePooling1D())
model.add(Dense(NUM_CLASSES, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.Accuracy(), 'binary_accuracy'])
model.fit(X, Y,
          batch_size=1,
          epochs=2,
          validation_data=(X, Y))

(22, 7) (22, 4)
Epoch 1/2
22/22 [==============================] - 0s 12ms/step - loss: 0.6911 - accuracy: 0.0000e+00 - binary_accuracy: 0.5227 - val_loss: 0.6858 - val_accuracy: 0.0000e+00 - val_binary_accuracy: 0.6250
Epoch 2/2
22/22 [==============================] - 0s 5ms/step - loss: 0.6842 - accuracy: 0.0000e+00 - binary_accuracy: 0.6705 - val_loss: 0.6805 - val_accuracy: 0.0000e+00 - val_binary_accuracy: 0.7159


In [131]:
y_pred = model.predict(X) 
y_true = Y

accuracy_score(y_true.ravel(), y_pred.ravel()>=0.5) 

0.6704545454545454

In [133]:
y_pred = model.predict(X) 
y_true = Y

accuracy_score(y_true.ravel(), y_pred.ravel()>=0.5) 


m = tf.keras.metrics.BinaryAccuracy()
m.update_state(y_true, y_pred>0.5)
print(m.result().numpy())

m = tf.keras.metrics.Accuracy()
m.update_state(y_true, y_pred>0.5)
print(m.result().numpy())

0.67045456
0.67045456


0.6590909090909091

In [134]:
y_true1 = y_true.argmax(axis=-1) 
y_pred1 = y_pred.argmax(axis=-1)
print(accuracy_score(y_true1, y_pred1))

0.5454545454545454


NameError: name 'ipytest' is not defined

In [136]:
'a b cd efg '.split()

['a', 'b', 'cd', 'efg']

In [139]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from tensorflow.keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.callbacks import Callback

In [161]:
letters = 'c b a 9 8 7 6 5 4 3 2 1 0'.split()
X_train = [ letters[i:]  for i in range(len(letters))]
X_test = ['a 9 8 1 0 x']

In [163]:
tokenizer = text.Tokenizer(num_words=10)
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)

X_train_transform = tokenizer.texts_to_sequences(X_train)
X_test_transform = tokenizer.texts_to_sequences(X_test)

# 可以看到编号为字符9被忽略了，因为它的index>=num_words
print(X_test_transform)

{'0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, 'a': 11, 'b': 12, 'c': 13}
[[9, 2, 1]]


{'0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 'j': 11,
 'i': 12,
 'h': 13,
 'g': 14,
 'f': 15,
 'e': 16,
 'd': 17,
 'c': 18,
 'b': 19,
 'a': 20}

In [164]:
!pip install joblib textblob -U

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 303 kB 309 kB/s eta 0:00:01
     |████████████████████████████████| 636 kB 3.1 MB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.0.0
    Uninstalling joblib-1.0.0:
      Successfully uninstalled joblib-1.0.0


In [178]:
from joblib import Parallel, delayed
from textblob import TextBlob
from textblob.translate import NotTranslated

import argparse
import os
import pandas as pd

NAN_WORD = "_NAN_"


def translate(sentence, language):
    if hasattr(sentence, "decode"):
        sentence = sentence.decode("utf-8")

    text = TextBlob(sentence)
    try:
        text = text.translate(to=language)
        text = text.translate(to="en")
    except NotTranslated:
        pass

    return str(text)


languages = ["es", "de", "fr", "pt", "ru", "ar"]
thread_count = 10


sentences = ['Due to the 4 months of purchase, I have problems with the ram memory.',
             'It was a very long process to solve my problem',
             'They took a long time to communicate once the action plan was defined.'
            ]

results = {'en':sentences}


parallel = Parallel(thread_count, backend="threading", verbose=5)
for language in languages:
    print('Translate comments using "{0}" language'.format(language))
    translated_data = parallel(delayed(translate)(sentence, language) for sentence in sentences)
    results[language] = translated_data


Translate comments using "es" language


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of   3 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done   3 out of   3 | elapsed:    1.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


Translate comments using "de" language


[Parallel(n_jobs=10)]: Done   3 out of   3 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=10)]: Done   3 out of   3 | elapsed:    0.9s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


Translate comments using "fr" language


...

[Parallel(n_jobs=10)]: Done   3 out of   3 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=10)]: Done   3 out of   3 | elapsed:    0.9s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


Translate comments using "zh" language


[Parallel(n_jobs=10)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=10)]: Done   3 out of   3 | elapsed:    0.7s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


Translate comments using "ar" language


[Parallel(n_jobs=10)]: Done   3 out of   3 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done   3 out of   3 | elapsed:    1.0s finished


In [179]:
results

{'en': ['Due to the 4 months of purchase, I have problems with the ram memory.',
  'It was a very long process to solve my problem',
  'They took a long time to communicate once the action plan was defined.'],
 'es': ['Due to the 4 months of purchase, I have problems with the RAM.',
  'It was a very long process to solve my problem.',
  'They took a long time to communicate once the action plan was defined.'],
 'de': ['Due to the 4 month purchase I have problems with the RAM memory.',
  'It was a very long process to solve my problem',
  'Communication took a long time after the action plan was set.'],
 'fr': ['Due to the 4 months of purchase, I am having problems with the RAM memory.',
  'It was a very long process to resolve my issue',
  'They took a long time to communicate once the action plan was defined.'],
 'pt': ['Due to the 4 months of purchase, I am having problems with ram.',
  'It was a very long process to solve my problem',
  'They took a long time to communicate after th

In [175]:
df_results = pd.DataFrame.from_dict(results)
df_results

,en,es,de,fr,pt,ru
0,"Due to the 4 months of purchase, I have proble...","Due to the 4 months of purchase, I have proble...",Due to the 4 month purchase I have problems wi...,"Due to the 4 months of purchase, I am having p...","Due to the 4 months of purchase, I am having p...","Due to 4 months of purchase, I have problems w..."
1,It was a very long process to solve my problem,It was a very long process to solve my problem.,It was a very long process to solve my problem,It was a very long process to resolve my issue,It was a very long process to solve my problem,The solution to my problem was very long
2,They took a long time to communicate once the ...,They took a long time to communicate once the ...,Communication took a long time after the actio...,They took a long time to communicate once the ...,They took a long time to communicate after the...,"After the action plan was determined, it took ..."


In [197]:
import pandas as pd
df = pd.DataFrame({'num_legs': [2, 4, 8, 0, 6],
                   'num_wings': [2, 0, 0, 0, 2],
                   'num_specimen_seen': [10, 2, 1, 8, 6]},
                  index=['falcon', 'dog', 'spider', 'fish', 'white ant'])
display(df)

def test(df1):
    def test_(index, row):
        df1.loc[index].num_legs=df1.loc[index].num_legs*2
#         row.num_legs = row.num_legs*2
        return 1
    [test_(index, row) for index, row in df1.iterrows()]        
    return df1
    
    
display(test(df[0:3]))
display(df)

,num_legs,num_wings,num_specimen_seen
falcon,2,2,10
dog,4,0,2
spider,8,0,1
fish,0,0,8
white ant,6,2,6


,num_legs,num_wings,num_specimen_seen
falcon,4,2,10
dog,8,0,2
spider,16,0,1


,num_legs,num_wings,num_specimen_seen
falcon,4,2,10
dog,8,0,2
spider,16,0,1
fish,0,0,8
white ant,6,2,6


In [198]:
100 % 3

1

In [199]:
from sklearn.utils.class_weight import compute_class_weight
 
class_weight = 'balanced'
label = [0] * 9 + [1]*1 + [2, 2]
print(label) # [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2]
classes=[0, 1, 2]
weight = compute_class_weight(class_weight, classes, label)
print(weight) #[ 0.44444444 4.         2.        ]
print(.44444444 * 9) # 3.99999996
print(4 * 1) # 4
print(2 * 2) # 4


[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2]
[0.44444444 4.         2.        ]
3.99999996
4
4


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: FutureWarning:

Pass classes=[0, 1, 2], y=[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error



In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import models, layers, initializers

physical_devices = tf.config.experimental.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

def get_model(rnn_layer):
    inputs = layers.Input(shape=(3, 4))
    x = rnn_layer(inputs)
    model = models.Model(inputs=inputs, outputs=x)
    return model

def predict(rnn_layer, data):
    outputs = get_model(rnn_layer).predict(data)
    print('-'*40, type(rnn_layer).__name__, '-'*40)
    if isinstance(outputs, list):  
        for output in outputs:
            print(output, output.shape)
    else:
        print(outputs, outputs.shape)
    
np.random.seed(2021)
data = np.random.rand(1, 3, 4)
print(data) 

[[[0.60597828 0.73336936 0.13894716 0.31267308]
  [0.99724328 0.12816238 0.17899311 0.75292543]
  [0.66216051 0.78431013 0.0968944  0.05857129]]]


In [2]:
rnn_layer = layers.LSTM(units=2, 
                        kernel_initializer = initializers.RandomNormal(seed=2021),
                        recurrent_initializer = initializers.RandomNormal(seed=2021)
                       )
predict(rnn_layer, data)

rnn_layer = layers.GRU(units=2, 
                        kernel_initializer = initializers.RandomNormal(seed=2021),
                        recurrent_initializer = initializers.RandomNormal(seed=2021)
                       )
predict(rnn_layer, data)

---------------------------------------- LSTM ----------------------------------------
[[-0.00038722  0.02867693]] (1, 2)
---------------------------------------- GRU ----------------------------------------
[[ 0.00614149 -0.01545692]] (1, 2)


In [3]:
rnn_layer = layers.LSTM(units=2, 
                        kernel_initializer = initializers.RandomNormal(seed=2021),
                        recurrent_initializer = initializers.RandomNormal(seed=2021),
                        return_sequences=True)
predict(rnn_layer, data)

rnn_layer = layers.GRU(units=2, 
                       kernel_initializer = initializers.RandomNormal(seed=2021),
                       recurrent_initializer = initializers.RandomNormal(seed=2021),
                       return_sequences=True)
predict(rnn_layer, data)

---------------------------------------- LSTM ----------------------------------------
[[[-0.00172656  0.00940116]
  [ 0.01072936  0.04589876]
  [-0.00038722  0.02867693]]] (1, 3, 2)
---------------------------------------- GRU ----------------------------------------
[[[ 0.00662032 -0.01183234]
  [-0.0032729  -0.00783275]
  [ 0.00614149 -0.01545692]]] (1, 3, 2)


In [4]:
rnn_layer = layers.LSTM(units=2, 
                        kernel_initializer = initializers.RandomNormal(seed=2021),
                        recurrent_initializer = initializers.RandomNormal(seed=2021),
                        return_state=True)
predict(rnn_layer, data)

rnn_layer = layers.GRU(units=2, 
                       kernel_initializer = initializers.RandomNormal(seed=2021),
                       recurrent_initializer = initializers.RandomNormal(seed=2021),
                       return_state=True)
predict(rnn_layer, data)

---------------------------------------- LSTM ----------------------------------------
[[-0.00038722  0.02867693]] (1, 2)
[[-0.00038722  0.02867693]] (1, 2)
[[-0.00079747  0.05748591]] (1, 2)
---------------------------------------- GRU ----------------------------------------
[[ 0.00614149 -0.01545692]] (1, 2)
[[ 0.00614149 -0.01545692]] (1, 2)


In [2]:
rnn_layer = layers.LSTM(units=2, 
                        kernel_initializer = initializers.RandomNormal(seed=2021),
                        recurrent_initializer = initializers.RandomNormal(seed=2021),
                        return_sequences=True,
                        return_state=True)
predict(rnn_layer, data)

rnn_layer = layers.GRU(units=2, 
                       kernel_initializer = initializers.RandomNormal(seed=2021),
                       recurrent_initializer = initializers.RandomNormal(seed=2021),
                       return_sequences=True,
                       return_state=True)
predict(rnn_layer, data)

---------------------------------------- LSTM ----------------------------------------
[[[-0.00172656  0.00940116]
  [ 0.01072936  0.04589876]
  [-0.00038722  0.02867693]]] (1, 3, 2)
[[-0.00038722  0.02867693]] (1, 2)
[[-0.00079747  0.05748591]] (1, 2)
---------------------------------------- GRU ----------------------------------------
[[[ 0.00662032 -0.01183234]
  [-0.0032729  -0.00783275]
  [ 0.00614149 -0.01545692]]] (1, 3, 2)
[[ 0.00614149 -0.01545692]] (1, 2)


In [6]:
rnn_layer = layers.Bidirectional(layers.LSTM(units=2, 
                        kernel_initializer = initializers.RandomNormal(seed=2021),
                        recurrent_initializer = initializers.RandomNormal(seed=2021),
                        return_sequences=True))
predict(rnn_layer, data)

rnn_layer = layers.Bidirectional(layers.GRU(units=2, 
                       kernel_initializer = initializers.RandomNormal(seed=2021),
                       recurrent_initializer = initializers.RandomNormal(seed=2021),
                       return_sequences=True))
predict(rnn_layer, data)

---------------------------------------- Bidirectional ----------------------------------------
[[[-0.00172656  0.00940116  0.00276289  0.03511165]
  [ 0.01072936  0.04589876  0.00661553  0.03663033]
  [-0.00038722  0.02867693 -0.00785343 -0.00329136]]] (1, 3, 4)
---------------------------------------- Bidirectional ----------------------------------------
[[[ 0.00662032 -0.01183234  0.00526874 -0.01576701]
  [-0.0032729  -0.00783275 -0.00273875 -0.00760575]
  [ 0.00614149 -0.01545692  0.00773718 -0.01140916]]] (1, 3, 4)


In [11]:
outputs = get_model(rnn_layer).predict(data)
print(outputs)
outputs[:,-1,:]

[[[ 0.00662032 -0.01183234  0.00526874 -0.01576701]
  [-0.0032729  -0.00783275 -0.00273875 -0.00760575]
  [ 0.00614149 -0.01545692  0.00773718 -0.01140916]]]


array([[ 0.00614149, -0.01545692,  0.00773718, -0.01140916]],
      dtype=float32)

In [12]:
dataset_path = '/tf/eipi10/jian-xu3/snps_classification/models/{}'.format('snps_18510_c8')
logs_path = '{}/hpt/{}'.format(dataset_path, 'pg_snps')
! tensorboard --logdir {logs_path} --bind_all

2021-04-14 21:38:30.724251: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
TensorBoard 2.3.0 at http://3b132921d8c9:6006/ (Press CTRL+C to quit)
E0414 21:44:18.235365 140501210752768 directory_watcher.py:196] Unable to get size of /tf/eipi10/jian-xu3/snps_classification/models/snps_18510_c8/hpt/pg_snps/events.out.tfevents.1618404832.3b132921d8c9.31102.989.v2: /tf/eipi10/jian-xu3/snps_classification/models/snps_18510_c8/hpt/pg_snps/events.out.tfevents.1618404832.3b132921d8c9.31102.989.v2; No such file or directory
W0414 21:44:18.239319 140501210752768 plugin_event_multiplexer.py:263] Deleting accumulator 'pg_19'
W0414 21:44:18.240098 140501210752768 plugin_event_multiplexer.py:263] Deleting accumulator 'pg_3'
W0414 21:44:18.240203 140501210752768 plugin_event_multiplexer.py:263] Deleting accumulator 'pg_5'
W0414 21:44:18.240287 140501210752768 plugin_event_multiplexer.py:263] Deleting accumulator 'pg_23'
W0414 21:44:

In [13]:
! tensorboard --logdir /tf/eipi10/jian-xu3/snps_classification/models/snps_18510_c8/hpt/snps_18510_c8_hpt_pg_snps_0413 --bind_all

2021-04-15 12:37:19.343900: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
TensorBoard 2.3.0 at http://3b132921d8c9:6006/ (Press CTRL+C to quit)
^C


In [28]:
import os 
dataset_name='snps_18510_c8'
dataset_path='/tf/eipi10/jian.xu3/snps-classification/models/snps_18510_c8'
path = '/home/grid/eipi10/jian.xu3/snps-classification/models/snps_18510_c8/abcde/eee.vector'

In [31]:
def get_safe_file_path(path):
    if not os.path.exists(path):
        index = path.index(dataset_name) + len(dataset_name) + 1
        path = os.path.join(dataset_path, path[index:])
        return path
    else:
        return path

In [32]:
get_safe_file_path(path)

'/tf/eipi10/jian.xu3/snps-classification/models/snps_18510_c8/abcde/eee.vector'

In [22]:
path.index('snps_18510_c8')

54

In [23]:
path.index(dataset_name)

ValueError: substring not found

In [24]:
dataset_name

'/tf/eipi10/jian.xu3/snps-classification/models/snps_18510_c8'

In [37]:
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.client_context import ClientContext

hostname = 'https://hp.sharepoint.com'
server_relative_path = '/teams/CSGITRPA/RPADev'
input_folder = '/Shared Documents/snps_classification/input'
site_url = hostname + server_relative_path

document_client_id = 'c9a0e93c-8592-4bfd-b9c3-08cc93480ab2'
document_client_secret = 'coL0KKIJUZBCQP9gDrThcr7V3xVl3kA4CZj2Be/cj7g=' 

credentials = ClientCredential(document_client_id, document_client_secret)
document_client_context = ClientContext(site_url).with_credentials(credentials)

folder = document_client_context.web.get_folder_by_server_relative_url(input_folder)

document_client_context.load(folder)
document_client_context.execute_query()

print(len(folder.files))

0


In [14]:
print("\033[0;31m%s\033[0m" % "输出红色字符")     # 标准写法
print("\033[31m%s\033[0m" % "输出红色字符")       # 显示方式为0时，可以省略
print("\033[31m%s" % "输出红色字符")
print("输出红色字符")                             # 由于上面没有结尾，继续按照上面的输出
print("\033[31m%s\033[0m" % "输出红色字符")
print("恢复默认输出")                             # 恢复默认输出

输出红色字符
输出红色字符
输出红色字符
输出红色字符
输出红色字符
恢复默认输出


In [6]:
print("\033[0;31m%s\033[0m" % "输出红色字符")

输出红色字符


In [16]:
print("\033[0m你好麽，\033[1m我很好。")
print("\033[30;1m你好麽，\033[31;1m我很好。")
print("\033[32;1m你好麽，\033[1m我很好。")
print("\033[33;1m你好麽，\033[34;1m我很好。")
print("\033[35;1m你好麽，\033[36;1m我很好。")
print("\033[37;1m你好麽，\033[38;1m我很好。")

print("\033[41;1m你好麽，\033[0m我很好。")
print("\033[42;1m你好麽，\033[0m我很好。")
print("\033[43;1m你好麽，\033[0m我很好。")
print("\033[44;1m你好麽，\033[0m我很好。")
print("\033[45;1m你好麽，\033[0m我很好。")
print("\033[46;1m你好麽，\033[0m我很好。")
print("\033[47;1m你好麽，\033[0m我很好。")

你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
你好麽，我很好。
